In [1]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

from scipy.integrate import solve_ivp
from scipy.integrate import cumulative_trapezoid as cumtrapz
from scipy.interpolate import interp1d

import copernicusmarine

In [2]:
t = dynamicsymbols._t


x, y, z, theta, phi = dynamicsymbols("x y z theta phi")  # x and y -> buoy coordinates

m_b, m_d, l, g = sp.symbols(
    "m_b m_d l g", positive=True
)  # mass buoy, mass drogue in [kg], wire length in [m], g in [m(s^2)]

k_b, k_d = sp.symbols("k_b k_d", positive=True)  # drag coefficients for buoy and drogue

U_b, V_b, U_d, V_d = sp.symbols(
    "U_b V_b U_d V_d", real=True
)  # surface horizontal velocities

# U_0, V_0 = sp.symbols('U_0 V_0', real = True) # surface horizontal velocities

In [3]:
lon0, lat0 = 54.6, 10.5  # largest variance of shear between 0 and 3m depth

time0 = np.datetime64("2009-04-08T23:58:00")

In [4]:
def convert_to_lonlat(y_in_m, x_in_m, lon0=lon0, lat0=lat0):
    deg_per_meter_lon = (
        1 / 111000 / np.cos(np.deg2rad(lat0))
    )  # stole this from our pherwiss project
    deg_per_meter_lat = 1 / 111000

    lon = lon0 + x_in_m * deg_per_meter_lon
    lat = lat0 + y_in_m * deg_per_meter_lat

    return lon, lat

In [5]:
r_b = sp.Matrix([x, y, 0])  # buoy position

r = l * sp.Matrix(
    [
        sp.sin(theta) * sp.cos(phi),
        sp.sin(theta) * sp.sin(phi),
        sp.cos(theta),
    ]
)

r_d = r_b + r  # drogue position

display(r_d)

Matrix([
[l*sin(theta(t))*cos(phi(t)) + x(t)],
[l*sin(phi(t))*sin(theta(t)) + y(t)],
[                   l*cos(theta(t))]])

In [6]:
# velocities

v_b = r_b.diff(t)
v_d = r_d.diff(t)

u_b = sp.Matrix([U_b, V_b, 0])
u_d = sp.Matrix([U_d, V_d, 0])

In [7]:
def _mag(vec):
    return sp.sqrt(vec.dot(vec))


F_b = -k_b * _mag(v_b - u_b) * (v_b - u_b)  # drag buoy with ext. velocity
F_d = -k_d * _mag(v_d - u_d) * (v_d - u_d)  # drag drogue with ext. velocity

In [8]:
# lagrangian

T = sp.Rational(1, 2) * m_b * v_b.dot(v_b) + sp.Rational(1, 2) * m_d * v_d.dot(
    v_d
)  # kinetic energy

V = m_d * g * r_d[2]  # potential energy

L = T - V  # lagrangian

In [9]:
q = sp.Matrix([x, y, theta, phi])
qd = q.diff(t)
qdd = qd.diff(t)

In [10]:
Q = sp.Matrix([r_b.diff(qi).dot(F_b) + r_d.diff(qi).dot(F_d) for qi in q])

Q = sp.simplify(Q)

In [11]:
eoms = sp.Matrix(
    [L.diff(qdj).diff(t) - L.diff(qj) - Qj for qj, qdj, Qj in zip(q, qd, Q)]
)
eoms = sp.simplify(eoms)

In [12]:
M, F = sp.linear_eq_to_matrix(eoms, list(qdd))

In [13]:
par_syms = (m_b, m_d, l, g, k_b, k_d)

In [14]:
M_func = sp.lambdify(
    (t, x, y, theta, phi, *qd, m_b, m_d, l, g, k_b, k_d, U_b, V_b, U_d, V_d),
    M,
    modules="numpy",
)
F_func = sp.lambdify(
    (t, x, y, theta, phi, *qd, m_b, m_d, l, g, k_b, k_d, U_b, V_b, U_d, V_d),
    F,
    modules="numpy",
)

In [15]:
r_b_ = sp.lambdify((t,) + tuple(q) + tuple(qd) + par_syms + (U_b, V_b, U_d, V_d), r_b)
r_d_ = sp.lambdify((t,) + tuple(q) + tuple(qd) + par_syms + (U_b, V_b, U_d, V_d), r_d)

M_ = sp.lambdify((t,) + tuple(q) + tuple(qd) + par_syms, M)
F_ = sp.lambdify((t,) + tuple(q) + tuple(qd) + par_syms, F)

In [16]:
par_dict = {m_b: 0.5, m_d: 0.5, l: 3.0, g: 9.81, k_b: 0.5, k_d: 2.0}
par_vals = tuple(par_dict[k] for k in par_syms)

In [17]:
ds = copernicusmarine.open_dataset(
    dataset_id="cmems_mod_bal_phy_my_P1D-m",  # found via "Data Access" in the dataset view https://data.marine.copernicus.eu/product/BALTICSEA_MULTIYEAR_PHY_003_011/description
    service="arco-time-series",
)

ds_subset = ds.sel(time="2009-04", longitude=slice(9, 11), latitude=slice(54, 55))
ds_subset = ds_subset[["uo", "vo"]]
ds_subset = ds_subset.load()
ds_subset
# ds = xr.open_dataset("01_baltic_sea_particle_drift_flow.nc")
# ds

INFO - 2025-12-31T15:19:03Z - Selected dataset version: "202303"
INFO - 2025-12-31T15:19:03Z - Selected dataset part: "default"


<xarray.Dataset> Size: 57MB
Dimensions:    (time: 30, depth: 56, latitude: 60, longitude: 71)
Coordinates:
  * time       (time) datetime64[ns] 240B 2009-04-01 2009-04-02 ... 2009-04-30
  * depth      (depth) float32 224B 0.5016 1.516 2.548 ... 663.9 688.0 712.0
  * latitude   (latitude) float32 240B 54.01 54.02 54.04 ... 54.96 54.97 54.99
  * longitude  (longitude) float32 284B 9.042 9.069 9.097 ... 10.93 10.96 10.99
Data variables:
    uo         (time, depth, latitude, longitude) float32 29MB nan nan ... nan
    vo         (time, depth, latitude, longitude) float32 29MB nan nan ... nan
Attributes:
    references:   https://marine.copernicus.eu/
    title:        CMEMS NEMO daily integrated model fields
    comment:      Data on cropped native product grid. Horizontal velocities ...
    institution:  Baltic MFC, PU Danish Meteorological Institute
    source:       CMEMS BAL MFC NEMO model output converted to NetCDF
    contact:      servicedesk.cmems@mercator-ocean.eu
    Conventions:  CF-1.0

In [18]:
def get_uv(
    t, z_d, y, x, ds
):  # sadly I couldn't figure ot how to do this without loading the dataset four times.
    """Get horizontal currents at and drogue and buoy level."""

    time = time0 + t * np.timedelta64(1000, "ms")

    lon, lat = convert_to_lonlat(y_in_m=y, x_in_m=x)
    # not shown: conversion of (x,y) --> (lon,lat)

    # TODO: Note that this will load separately four times.
    # better: get ds_water_colum_at_time = ds.sel(time=..., latitude=..., longitude=...).load()
    # and then select U_b etc from there justing with depth=...

    U_b = (
        ds.sel(
            time=time,
            depth=0,
            latitude=lat,
            longitude=lon,
            method="nearest",
        )
        .uo.load()
        .data[()]
    )
    # load actually gets the data, .data[()] converts singleton xarray object to single number

    U_d = (
        ds.sel(
            time=time,
            depth=-z_d,
            latitude=lat,
            longitude=lon,
            method="nearest",
        )
        .uo.load()
        .data[()]
    )

    V_b = (
        ds.sel(
            time=time,
            depth=0,
            latitude=lat,
            longitude=lon,
            method="nearest",
        )
        .vo.load()
        .data[()]
    )

    V_d = (
        ds.sel(
            time=time,
            depth=-z_d,
            latitude=lat,
            longitude=lon,
            method="nearest",
        )
        .vo.load()
        .data[()]
    )

    return U_b, V_b, U_d, V_d

In [19]:
Ux, Uy = sp.symbols("Ux Uy", real=True)
subst = {
    x.diff(t): Ux,
    y.diff(t): Uy,
    theta.diff(t): 0,
    phi.diff(t): 0,
    x.diff(t, 2): 0,
    y.diff(t, 2): 0,
    theta.diff(t, 2): 0,
    phi.diff(t, 2): 0,
}
eoms_steady_state = sp.simplify(eoms.subs(subst))
display(eoms_steady_state)

Matrix([
[                                              -k_b*(U_b - Ux)*sqrt((U_b - Ux)**2 + (Uy - V_b)**2) - k_d*(U_d - Ux)*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)],
[                                               k_b*(Uy - V_b)*sqrt((U_b - Ux)**2 + (Uy - V_b)**2) + k_d*(Uy - V_d)*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)],
[-l*(g*m_d*sin(theta(t)) + k_d*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)*(U_d*cos(phi(t)) - Ux*cos(phi(t)) - Uy*sin(phi(t)) + V_d*sin(phi(t)))*cos(theta(t)))],
[                         k_d*l*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)*(U_d*sin(phi(t)) - Ux*sin(phi(t)) + Uy*cos(phi(t)) - V_d*cos(phi(t)))*sin(theta(t))]])

In [20]:
th, ph = sp.symbols("th ph", real=True)
parameter = {k_b: 3.0, k_d: 2.0, l: 3.0, g: 9.81, m_d: 1.0}
eoms_subs = eoms_steady_state.subs({theta: th, phi: ph}).subs(parameter)

eoms_subs.free_symbols

{U_b, U_d, Ux, Uy, V_b, V_d, ph, th}

In [21]:
equations = sp.Matrix(eoms_subs)
vars_ = (Ux, Uy, th, ph)

In [22]:
guess = [0.0, 0.0, 0.1, 1.0]

In [23]:
def solve_steady(ti, x, y, guess, ds_subset):
    th_guess = guess[2]
    z_d = max(0.0, float(parameter[l]) * np.cos(float(th_guess)))
    Ub, Vb, Ud, Vd = get_uv(ti, z_d, y, x, ds_subset)
    eq_num = eoms_subs.subs({U_b: Ub, V_b: Vb, U_d: Ud, V_d: Vd})
    sol = sp.nsolve(eq_num, vars_, sp.Matrix(guess))
    sol_array = np.array(sol)
    return sol_array